In [1]:
# Run in terminal or command prompt
# python3 -m spacy download en

import numpy as np
import pandas as pd
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet as wn
import re, nltk, spacy, gensim, string

# Sklearn
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint

# Plotting tools
import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
%matplotlib inline

C:\Users\driss\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
# empty doc list
doc_complete = []

with open (".\\data\\stackoverflow_vacature_tekst2.csv", "r") as myfile:
    for line in myfile:
        doc_complete.append(line)

In [3]:
tech_words_set = set()

with open(".\\data\\top50000_tags_short2.csv", "r") as myfiletags:
    for line in myfiletags:
        #tech_words_set.add(line.replace('"',''))
        tech_words_set.add(line.rstrip())
        
#for word in tech_words_set:
#    print(word)


In [4]:
stop = set(stopwords.words(['english', 'german', 'dutch']))
#stop_de = set(stopwords.words('german'))
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()
def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    #stop_de_free = " ".join([i for i in stop_free.lower().split() if i not in stop_de])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    #print(normalized)
    #[print(i) for i in normalized.lower().split()]
#     normalized2 = " ".join([i for i in normalized.lower().split() if i in tech_words_set])
    #print(normalized2)
    return normalized
    ####return normalized

doc_clean = [clean(doc).split() for doc in doc_complete] 

In [5]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# Run in terminal: python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only Noun, Adj, Verb, Adverb
data_lemmatized = lemmatization(doc_clean, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])


In [6]:
vectorizer = CountVectorizer(analyzer='word',       
                             min_df=10,                        # minimum reqd occurences of a word 
                             stop_words=['english','german'],  # remove stop words
                             lowercase=True,                   # convert all words to lowercase
                             token_pattern='[a-zA-Z0-9]{1,}',  # num chars > 3
                             # max_features=50000,             # max number of uniq words
                            )

data_vectorized = vectorizer.fit_transform(data_lemmatized)

In [7]:
# Materialize the sparse data
data_dense = data_vectorized.todense()

# Compute Sparsicity = Percentage of Non-Zero cells
print("Sparsicity: ", ((data_dense > 0).sum()/data_dense.size)*100, "%")

Sparsicity:  3.41104511611 %


In [8]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# Run in terminal: python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only Noun, Adj, Verb, Adverb
data_lemmatized = lemmatization(doc_clean, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
# print(data_lemmatized[:2])

In [9]:
# Build LDA Model
lda_model = LatentDirichletAllocation(n_components=25,           # Number of topics
                                      max_iter=10,               # Max learning iterations
                                      learning_method='online',   
                                      random_state=100,          # Random state
                                      batch_size=128,            # n docs in each learning iter
                                      evaluate_every = -1,       # compute perplexity every n iters, default: Don't
                                      n_jobs = -1,               # Use all available CPUs
                                     )
lda_output = lda_model.fit_transform(data_vectorized)

# print(lda_model)  # Model attributes

In [10]:
# Log Likelyhood: Higher the better
print("Log Likelihood: ", lda_model.score(data_vectorized))

# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
print("Perplexity: ", lda_model.perplexity(data_vectorized))

# See model parameters
# pprint(lda_model.get_params())

Log Likelihood:  -4499313.0999
Perplexity:  903.096079236


In [11]:
# Define Search Param
search_params = {'n_components': [22], 'learning_decay': [.5, .7, .9]}

# Init the Model
lda = LatentDirichletAllocation()

# Init Grid Search Class
model = GridSearchCV(lda, param_grid=search_params,cv=5)

# Do the Grid Search
model.fit(data_vectorized)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='batch', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_components=10, n_jobs=None, n_topics=None, perp_tol=0.1,
             random_state=None, topic_word_prior=None,
             total_samples=1000000.0, verbose=0),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_components': [22], 'learning_decay': [0.5, 0.7, 0.9]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [12]:
# Best Model
best_lda_model = model.best_estimator_

# Model Parameters
print("Best Model's Params: ", model.best_params_)

# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)

# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(data_vectorized))

Best Model's Params:  {'learning_decay': 0.9, 'n_components': 22}
Best Log Likelihood Score:  -981325.701718
Model Perplexity:  883.388696385


In [13]:
# Create Document - Topic Matrix
lda_output = best_lda_model.transform(data_vectorized)

# column names
topicnames = ['Topic' + str(i) for i in range(best_lda_model.n_components)]

# index names
docnames = ["Doc" + str(i) for i in range(len(doc_clean))]

# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic

# Styling
def color_green(val):
    color = 'green' if val > .1 else 'black'
    return 'color: {col}'.format(col=color)

def make_bold(val):
    weight = 700 if val > .1 else 400
    return 'font-weight: {weight}'.format(weight=weight)

# Apply Style
df_document_topics = df_document_topic.head(100).style.applymap(color_green).applymap(make_bold)
df_document_topics

In [14]:
df_topic_distribution = df_document_topic['dominant_topic'].value_counts().reset_index(name="Num Documents")
df_topic_distribution.columns = ['Topic Num', 'Num Documents']
df_topic_distribution

,Topic Num,Num Documents
0,15,866
1,0,299
2,5,290
3,9,283
4,8,223
5,14,170
6,3,122
7,7,114
8,10,98
9,20,63


In [15]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(best_lda_model, data_vectorized, vectorizer, mds='tsne')
panel

PreparedData(topic_coordinates=            Freq  cluster  topics           x           y
topic                                                    
15     21.294677        1       1 -107.349945 -226.409058
0      10.989584        1       2 -116.974312  -37.854115
5      10.585566        1       3  -43.132164   38.201092
8       8.182438        1       4 -113.945686  223.459534
14      6.404871        1       5   68.332855 -134.500320
19      5.911035        1       6  173.044540  211.187317
7       5.817764        1       7  182.178940 -161.668930
10      5.035330        1       8  242.794510  -40.677757
3       4.416200        1       9   56.595589   49.119656
9       3.156007        1      10 -197.042068 -124.738228
20      3.119616        1      11  265.763306   97.382950
11      2.534837        1      12  159.048523   67.842911
12      1.884785        1      13   20.990057  260.844055
17      1.810413        1      14   11.995614  -46.341286
21      1.573881        1      15 -134.148621   83.310707
16      1.557954        1      16 -227.563370  144.425369
13      1.529409        1      17  -52.915421 -128.494614
2       1.499960        1      18 -233.214539    2.751799
1       1.104117        1      19   76.768082  153.422806
4       0.824027        1      20  -31.009827  143.597168
18      0.580215        1      21  126.321251  -39.758129
6       0.187312        1      22   33.237309 -240.729538, topic_info=     Category          Freq            Term         Total  loglift  logprob
term                                                                       
3629  Default   1424.000000        security   1424.000000  30.0000  30.0000
1511  Default  12145.000000      experience  12145.000000  29.0000  29.0000
1006  Default   3491.000000           datum   3491.000000  28.0000  28.0000
4006  Default   4029.000000          system   4029.000000  27.0000  27.0000
2773  Default   1199.000000         network   1199.000000  26.0000  26.0000
4041  Default  10591.000000            team  10591.000000  25.0000  25.0000
3775  Default   5905.000000        software   5905.000000  24.0000  24.0000
4536  Default   9908.000000            work   9908.000000  23.0000  23.0000
1110  Default   6068.000000     development   6068.000000  22.0000  22.0000
563   Default   1006.000000            bull   1006.000000  21.0000  21.0000
4065  Default   5792.000000      technology   5792.000000  20.0000  20.0000
239   Default   4173.000000     application   4173.000000  19.0000  19.0000
3201  Default   4967.000000         product   4967.000000  18.0000  18.0000
1753  Default    824.000000          fuumlr    824.000000  17.0000  17.0000
1108  Default   2173.000000       developer   2173.000000  16.0000  16.0000
4085  Default   2261.000000            test   2261.000000  15.0000  15.0000
3885  Default    658.000000          status    658.000000  14.0000  14.0000
4439  Default   2182.000000             web   2182.000000  13.0000  13.0000
1082  Default   4527.000000          design   4527.000000  12.0000  12.0000
2258  Default   1292.000000             job   1292.000000  11.0000  11.0000
2781  Default   3523.000000             new   3523.000000  10.0000  10.0000
2385  Default   1961.000000           learn   1961.000000   9.0000   9.0000
2907  Default   1833.000000     opportunity   1833.000000   8.0000   8.0000
4054  Default   2931.000000       technical   2931.000000   7.0000   7.0000
3114  Default   1192.000000        position   1192.000000   6.0000   6.0000
3682  Default   3200.000000         service   3200.000000   5.0000   5.0000
2461  Default   2076.000000            look   2076.000000   4.0000   4.0000
737   Default    707.000000         comcast    707.000000   3.0000   3.0000
2501  Default   2081.000000      management   2081.000000   2.0000   2.0000
1107  Default   3446.000000         develop   3446.000000   1.0000   1.0000
...       ...           ...             ...           ...      ...      ...
995   Topic22      6.332724      daruumlber     19.290714

In [16]:
# Topic-Keyword Matrix
df_topic_keywords = pd.DataFrame(best_lda_model.components_)

# Assign Column and Index
df_topic_keywords.columns = vectorizer.get_feature_names()
df_topic_keywords.index = topicnames

# View
df_topic_keywords.head()

,2minute,2nd,3d,3rd,401k,6th,95b,aanleid,ab,abgeschlossen,...,zone,zookeeper,zos,zowel,zudem,zukunft,zurich,zusamman,zusammen,zusammenarbeit
Topic0,0.045455,0.045455,0.045455,0.490331,0.045455,0.045455,0.045455,0.045455,1.045455,0.045455,...,1.832087,0.045455,0.045455,0.045455,0.045455,0.045455,0.045455,0.045455,0.045455,0.045455
Topic1,0.045455,0.045455,0.045455,0.045455,0.045455,0.045455,0.045455,0.045455,0.045455,0.045455,...,0.045455,0.045455,0.045455,0.045455,0.045455,0.045455,0.045455,0.045455,0.045455,0.045455
Topic2,0.045455,0.045455,11.919791,0.045455,0.045455,0.045455,0.045455,0.045455,0.045455,0.045455,...,0.045455,0.045455,0.045455,0.045455,0.045455,0.045455,0.045455,0.045455,0.045455,0.045455
Topic3,0.045455,3.208648,0.045455,6.350231,0.045455,0.045455,0.045455,0.045455,1.045455,0.045455,...,1.693545,0.045455,0.045455,0.045455,0.045455,0.045455,0.045455,0.045455,0.045455,0.045455
Topic4,0.045455,1.790467,0.045455,0.045455,1.798848,0.045455,0.045455,0.045455,0.045455,0.045455,...,1.155639,0.045463,0.045455,0.045455,0.045455,0.045455,0.045455,0.045455,0.045455,0.045455


In [17]:
# Show top n keywords for each topic
def show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=20):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

topic_keywords = show_topics(vectorizer=vectorizer, lda_model=best_lda_model, n_words=15)        

# Topic - Keywords Dataframe
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14
Topic 0,work,team,make,company,people,good,help,get,look,learn,technology,new,day,experience,great
Topic 1,security,risk,network,information,vulnerability,incident,threat,control,compliance,cyber,cybersecurity,secure,policy,analysis,system
Topic 2,experience,hay,work,position,team,recruiter,benefit,dental,medical,interview,resume,write,opportunity,vision,information
Topic 3,development,work,web,experience,developer,application,javascript,team,software,new,knowledge,skill,technology,project,develop
Topic 4,network,linux,experience,work,protocol,position,center,configuration,system,networking,infrastructure,require,automation,php,look
Topic 5,experience,system,work,service,skill,support,technology,management,cloud,solution,team,application,technical,development,environment
Topic 6,t,system,region,siemen,fxc3xbcr,factory,mehr,beta,job,moumlchten,welt,software,gruppe,morgen,bereich
Topic 7,technology,team,solution,development,work,global,experience,build,design,amp,youxe2x80x99ll,world,business,financial,big
Topic 8,team,test,software,development,experience,product,testing,work,quality,project,technical,develop,skill,process,lead
Topic 9,fuumlr,team,sowie,kenntnisse,erfahrung,entwicklung,software,arbeiten,aufgaben,gute,suchen,hast,bieten,bereich,kunden
